Flower recognition uses the edge and colour characteristics of flower images to classify flowers. There are many species of flowers in the world. Some species have many colours, such as roses. It is difficult to remember all the names of flowers and their information. Additionally, someone may be confused with similar flower species.

For example, white champaka and champak have similar names and petal shapes, but they have different colours and petal lengths.

At present, it is almost impossible to identify any particular flower or flower species in any way other than to seek information based on personal knowledge and expert experience. The availability of such experts can be an obstacle to this search for information.

Searching for such information on the Internet today is very limited to keyword research; word processor. Even then, the searcher has to come up with sufficiently useful keywords, which they cannot do, which is the crux of the matter.

The dataset we are using here for the flower recognition contains 4242 flower images. Data collection is based on Flickr data, google images, Yandex images. We can use this data set to recognize the flowers in the photo.

The images are divided into five classes: chamomile, tulip, rose, sunflower, dandelion. For each class, there are approximately 800 photos. The photos are not in high resolution, approximately `320×240` pixels. Photos are not reduced to one size, they have different proportions.

In [1]:
import os
import cv2
import numpy as np

#Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Tensorflow Keras CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

#Plot Images
import matplotlib.pyplot as plt

In [2]:
folder_dir = 'flowers'

Now the next step is to read each image in the data and create a label for each with the name of the folder:

In [3]:
data = []
label = []

SIZE = 128 #Crop the image to 128x128

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir, folder)):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE,SIZE))
            data.append(im)
        else:
            continue

Now let’s convert the data into numerical values:

In [4]:
data_arr = np.array(data)
label_arr = np.array(label)

Now let’s use the Label encoder and normalize the data:

In [5]:
encoder = LabelEncoder()
y = encoder.fit_transform(label_arr)
y = to_categorical(y,5)
X = data_arr/255

The next step is to split the dataset into 80% training and 20% test sets:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

Now let’s build a neural network model for Flower Recognition:

In [7]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (SIZE,SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(5, activation = "softmax"))

Before compiling the model we need to create more training images to prevent overfitting:

In [8]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

In [9]:
datagen.fit(X_train)

Now let’s compile the neural network model:

In [10]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
batch_size=32
epochs=64
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),
                              verbose = 1)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/64
108/108 [==============================] - 1000s 8s/step - loss: 1.5616 - accuracy: 0.2796 - val_loss: 1.2831 - val_accuracy: 0.4803
Epoch 2/64
108/108 [==============================] - 1127s 10s/step - loss: 1.3204 - accuracy: 0.4317 - val_loss: 1.1818 - val_accuracy: 0.4907
Epoch 3/64
108/108 [==============================] - 1022s 9s/step - loss: 1.2606 - accuracy: 0.4630 - val_loss: 1.1751 - val_accuracy: 0.5185
Epoch 4/64
108/108 [==============================] - 973s 9s/step - loss: 1.2140 - accuracy: 0.5136 - val_loss: 1.0947 - val_accuracy: 0.5648
Epoch 5/64
108/108 [==============================] - 1012s 9s/step - loss: 1.1915 - accuracy: 0.5133 - val_loss: 1.0496 - val_accuracy: 0.5764
Epoch 6/64
108/108 [==============================] - 961s 9s/step - loss: 1.1552 - accuracy: 0.5382 - val_loss: 0.9631 - val_accuracy: 0.6319
Epoch 7/64
108/108 [==============================] - 942s 9s/step - loss: 1.0824 - accuracy: 0.5626 - val_loss: 0.9928 - val_accuracy: 0

Now let’s let the model if it recognize flowers properly:

In [ ]:
categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25, 40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')